# Reproduction

This notebook currently imports results from a run of PHC.py and attempts to map them to Table 6.

Table 6 results are from Shoaib M, Ramamohan V. **Simulation modeling and analysis of primary health center operations**. *SIMULATION* 98(3):183-208. (2022). <https://doi.org/10.1177/00375497211030931>.

## Set up

In [1]:
# To run model
import PHC
import numpy as np

# To import and process results
import pandas as pd
import xlrd
import os

In [2]:
table6_path = '../original_study/tab6.csv'

## Run model

Set parameters for each configuration based on Table 3.

In [3]:
# For all: days 365, warm up 180, replication 10, 6 inpatient beds, 1 delivery
# bed, 3 staff nurse, 1 NCD nurse. These are default parameters in the model,
# and below, have just specified the parameters that changed between
# configurations

# TODO: Change all replication to 100 (just running with 10 for now for speed)

t6_c1_param = {
    'doc_cap': 2,
    'OPD_iat': 4,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c1.xls',
}

t6_c2_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'delivery_iat': 2880,
    'ANC_iat': 2880,
    'rep_file': 't6_c2.xls',
}

t6_c3_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'any_delivery': False,
    'any_ANC': False,
    'rep_file': 't6_c3.xls'
}

t6_c4_param = {
    'doc_cap': 2,
    'OPD_iat': 3,
    'mean': 5,
    'sd': 1,
    'consult_boundary_1': 2,
    'consult_boundary_2': 2,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c4.xls'
}

Run model for each configuration

In [4]:
PHC.main(**{f's_{k}': v for k, v in t6_c4_param.items()})

 No of replications done 0
 No of replications done 1
 No of replications done 2
 No of replications done 3
 No of replications done 4
 No of replications done 5
 No of replications done 6
 No of replications done 7
 No of replications done 8
 No of replications done 9


In [ ]:
# Input configuration parameters to main() (have to append with 's_', and
# add ** as inputting dict as parameters)
# for config in [t6_c1_param, t6_c2_param, t6_c3_param, t6_c4_param]:
#     print(f'''Running: {config['rep_file']}''')
#     PHC.main(**{f's_{k}': v for k, v in config.items()})

## Import and process replication results

In [5]:
# Make dictionary with labels from table 6, and corresponding names from model output
t6_labels = {
  'doc occ': 'Doctor utilisation',
  'NCD occ': 'NCD Nurse utilisation',
  'staff nurse occ': 'Staff nurse utilisation',
  'pharm occ': 'Pharmacist utilisation',
  'lab occ': 'Lab utilisation',
  'ipd bed occ': 'Inpatient bed utilisation',
  'del occ': 'Labour bed utilisation',  # "Del" stands for delivery
  'OPD q len': 'Mean length of OPD queue (number of patients)',
  'OPD Q wt': 'OPD queue waiting time (minutes)',
  'pharmacy q len': 'Mean length of pharmacy queue (number of patients)',
  'Pharmacy Q wt': 'Pharmacy queue waiting time (minutes)',
  'lab q len': 'Mean length of Lab queue (number of patients)',
  'Lab Q wt': 'Lab queue waiting time (minutes)',
  'prop_del_referred': 'Fraction of childbirth cases referred'
}

In [6]:
# List of files to loop through
files = ['t6_c1', 't6_c2', 't6_c3', 't6_c4']

# Empty list to store results
result_list = []

for f in files:
    # Import .xls and convert to pandas dataframe
    book = xlrd.open_workbook(os.path.join('outputs', f'{f}.xls'))
    result = pd.read_excel(book, header=None, index_col=0)

    # Find mean from the replications
    av_result = result.mean(axis=1)

    # Calculate the proportion of childbirth cases referred and add to series
    prop_del_referred = pd.Series(
        av_result['del referred'] / av_result['Del patients'],
        index=['prop_del_referred'])
    new_av_result = pd.concat([av_result, prop_del_referred])

    # Convert to df, save as model config1, drop duplicates, and display
    res = (new_av_result
           .to_frame(name=f'model_{f}')
           #.reset_index()
           #.rename(columns= {'index': 'simple_outcome'})
           .drop_duplicates())

    result_list.append(res)

# Combine into single dataframe
summary = (pd.concat(result_list, axis=1)
           .reset_index()
           .rename(columns= {'index': 'model_outcome'}))

# Add labels to model results
summary['t6_outcome'] = summary['model_outcome'].map(t6_labels)

summary

/tmp/ipykernel_26989/1311786309.py:17: RuntimeWarning: invalid value encountered in double_scalars
  av_result['del referred'] / av_result['Del patients'],


,model_outcome,model_t6_c1,model_t6_c2,model_t6_c3,model_t6_c4,t6_outcome
0,OPD patients,33133.600000,14872.400000,14933.900000,44161.600000,NaN
1,IPD patients,184.000000,190.300000,181.600000,180.200000,NaN
2,ANC patients,371.200000,211.500000,0.000000,373.900000,NaN
3,Del patients,363.000000,176.900000,NaN,363.100000,NaN
4,OPD Q wt,0.009304,0.180556,0.034672,7.087377,OPD queue waiting time (minutes)
5,Pharmacy Q wt,1.019412,0.240693,0.231200,1.277731,Pharmacy queue waiting time (minutes)
6,Lab Q wt,2.087956,0.605414,0.570976,3.170211,Lab queue waiting time (minutes)
7,doc occ,0.269412,0.372110,0.355399,1.145836,Doctor utilisation
8,Lab patient list,189463.400000,84293.200000,86041.800000,254249.200000,NaN
9,OPD q len,0.008821,0.167379,0.034586,6.932062,Mean length of OPD queue (number of patients)


## Import table 6 results and compare against run results

In [7]:
# Import table 6
t6 = pd.read_csv(table6_path).rename(columns={'outcome': 't6_outcome'})
t6

,t6_outcome,config1_mean,config1_sd,config2_mean,config2_sd,config3_mean,config3_sd,benchmark_mean,benchmark_sd
0,Doctor utilisation,0.268,0.003,0.372,0.004,0.354,0.002,1.142,0.006
1,NCD Nurse utilisation,0.865,0.011,0.469,0.005,0.468,0.005,1.232,0.019
2,Staff nurse utilisation,0.323,0.008,0.243,0.006,0.160,0.001,0.322,0.008
3,Pharmacist utilisation,0.643,0.004,0.288,0.003,0.289,0.003,0.855,0.005
4,Lab utilisation,0.559,0.008,0.254,0.004,0.239,0.004,0.736,0.011
5,Inpatient bed utilisation,0.093,0.004,0.055,0.003,0.011,0.001,0.093,0.004
6,Labour bed utilisation,0.283,0.010,0.153,0.009,NaN,NaN,0.281,0.012
7,Mean length of OPD queue (number of patients),0.000,0.000,0.007,0.001,0.001,0.000,0.817,0.027
8,OPD queue waiting time (minutes),0.009,0.004,0.171,0.032,0.034,0.001,6.789,0.268
9,Mean length of pharmacy queue (number of patie...,0.090,0.002,0.010,0.001,0.009,0.000,0.150,0.002


In [8]:
# Merge
compare = t6.merge(summary)
compare.head()

,t6_outcome,config1_mean,config1_sd,config2_mean,config2_sd,config3_mean,config3_sd,benchmark_mean,benchmark_sd,model_outcome,model_t6_c1,model_t6_c2,model_t6_c3,model_t6_c4
0,Doctor utilisation,0.268,0.003,0.372,0.004,0.354,0.002,1.142,0.006,doc occ,0.269412,0.372110,0.355399,1.145836
1,NCD Nurse utilisation,0.865,0.011,0.469,0.005,0.468,0.005,1.232,0.019,NCD occ,0.869785,0.469595,0.470006,1.231760
2,Staff nurse utilisation,0.323,0.008,0.243,0.006,0.160,0.001,0.322,0.008,staff nurse occ,0.322609,0.242049,0.160632,0.321931
3,Pharmacist utilisation,0.643,0.004,0.288,0.003,0.289,0.003,0.855,0.005,pharm occ,0.642954,0.288769,0.289968,0.856621
4,Lab utilisation,0.559,0.008,0.254,0.004,0.239,0.004,0.736,0.011,lab occ,0.557508,0.253362,0.240857,0.738856


In [9]:
compare_col = [
    ('config1_mean', 'model_t6_c1'),
    ('config2_mean', 'model_t6_c2'),
    ('config3_mean', 'model_t6_c3'),
    ('benchmark_mean', 'model_t6_c4')]

for col in compare_col:
    # Get the two columns
    subset = compare[list(col)]
    # Find percent change
    pct_change = subset.pct_change(axis=1).iloc[:, 1]*100
    # Add back to compare dataframe
    compare[f'pct_change_{col[1]}'] = pct_change

# Display each of the results
for col in compare_col:
    # Set outcome as index, and get the two results plus percent change
    subset = compare.set_index('t6_outcome')[
        list(col) + [f'pct_change_{col[1]}']]
    display(subset)

,config1_mean,model_t6_c1,pct_change_model_t6_c1
t6_outcome,,,
Doctor utilisation,0.268,0.269412,0.526750
NCD Nurse utilisation,0.865,0.869785,0.553143
Staff nurse utilisation,0.323,0.322609,-0.121114
Pharmacist utilisation,0.643,0.642954,-0.007225
Lab utilisation,0.559,0.557508,-0.266928
Inpatient bed utilisation,0.093,0.093243,0.261063
Labour bed utilisation,0.283,0.287000,1.413428
Mean length of OPD queue (number of patients),0.000,0.008821,inf
OPD queue waiting time (minutes),0.009,0.009304,3.380654


,config2_mean,model_t6_c2,pct_change_model_t6_c2
t6_outcome,,,
Doctor utilisation,0.372,0.372110,0.029651
NCD Nurse utilisation,0.469,0.469595,0.126811
Staff nurse utilisation,0.243,0.242049,-0.391160
Pharmacist utilisation,0.288,0.288769,0.266990
Lab utilisation,0.254,0.253362,-0.251295
Inpatient bed utilisation,0.055,0.055268,0.487291
Labour bed utilisation,0.153,0.150000,-1.960784
Mean length of OPD queue (number of patients),0.007,0.167379,2291.127426
OPD queue waiting time (minutes),0.171,0.180556,5.588177


,config3_mean,model_t6_c3,pct_change_model_t6_c3
t6_outcome,,,
Doctor utilisation,0.354,0.355399,0.395336
NCD Nurse utilisation,0.468,0.470006,0.428706
Staff nurse utilisation,0.160,0.160632,0.395143
Pharmacist utilisation,0.289,0.289968,0.334861
Lab utilisation,0.239,0.240857,0.776935
Inpatient bed utilisation,0.011,0.011464,4.218362
Labour bed utilisation,NaN,NaN,NaN
Mean length of OPD queue (number of patients),0.001,0.034586,3358.558494
OPD queue waiting time (minutes),0.034,0.034672,1.975607


,benchmark_mean,model_t6_c4,pct_change_model_t6_c4
t6_outcome,,,
Doctor utilisation,1.142,1.145836,0.335941
NCD Nurse utilisation,1.232,1.231760,-0.019499
Staff nurse utilisation,0.322,0.321931,-0.021522
Pharmacist utilisation,0.855,0.856621,0.189635
Lab utilisation,0.736,0.738856,0.388032
Inpatient bed utilisation,0.093,0.093002,0.002285
Labour bed utilisation,0.281,0.278000,-1.067616
Mean length of OPD queue (number of patients),0.817,6.932062,748.477614
OPD queue waiting time (minutes),6.789,7.087377,4.395003
